### 랭체인 준비

In [1]:
# 패키지 설치
!pip install langchain==0.3.7
!pip install langchain-google-genai
!pip install langchain_community
!pip install langgraph

  Using cached langchain-0.3.7-py3-none-any.whl.metadata (7.1 kB)
Using cached langchain-0.3.7-py3-none-any.whl (1.0 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.14 requires langchain<0.4.0,>=0.3.14, but you have langchain 0.3.7 which is incompatible.
  Using cached langchain-0.3.14-py3-none-any.whl.metadata (7.1 kB)
Using cached langchain-0.3.14-py3-none-any.whl (1.0 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


In [2]:
import os
from google.colab import userdata

# 환경 변수 준비(좌측 상단의 열쇠 아이콘으로 GOOGLE_API_KEY 설정)
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

###랭스미스

In [3]:
import os
from uuid import uuid4

# 환경 변수 준비
unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

### 임베딩 모델 준비

In [4]:
# 허깅 페이스 패키지 설치
!pip install langchain-huggingface
!pip install nltk==3.9.1

  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.


In [5]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# 임베딩 모델 준비
hf_embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
# 임베딩 모델 실행
embeds = hf_embeddings.embed_query(
    "임베딩 벡터로 변환할 텍스트"
)
print(len(embeds))
print(embeds)

1024
[-0.04747262969613075, -0.010639168322086334, 0.014603265561163425, 0.011628430336713791, -0.0001373385894112289, -0.020408160984516144, 0.025050129741430283, -0.006968156900256872, 0.02665463648736477, 0.004971162416040897, 0.010392991825938225, -0.0043031079694628716, 0.041137877851724625, -0.004970446694642305, -0.014922318048775196, -0.011346534825861454, 0.010836995206773281, 0.009942078031599522, 0.033208854496479034, -0.02325483411550522, -0.027908815070986748, -0.058665692806243896, -0.04178787022829056, -0.004477020353078842, 0.002314687008038163, 0.0391729399561882, 0.06155860051512718, -0.011041290126740932, -0.008098301477730274, 0.00018581240146886557, 0.02594032697379589, 0.007040462456643581, 0.033704258501529694, -0.07545164972543716, -0.0017990806372836232, 0.0011119183618575335, 0.01171915140002966, -0.00626410311087966, -0.05732319504022598, 0.04886891692876816, 0.0024109533987939358, -0.022014090791344643, -0.01228233240544796, -0.003186675487086177, 0.02705607

# 벡터 스토어 준비

In [7]:
from langchain_core.documents import Document

# 문서 준비
documents = [
    Document(
        page_content="개는 충성심이 높고 친근하기 때문에 아주 멋진 반려동물입니다.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="고양이는 독립심이 높은 반려동물이며, 자기만의 공간을 만끽하는 경우가 많습니다.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="금붕어는 비교적 간단하게 키울 수 있기 때문에 초심자에게 인기가 많은 반려동물입니다.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="앵무새는 사람의 말을 따라할 수 있는 영리한 새입니다.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="토끼는 사회적인 동물이며, 자유롭게 뛰어다닐 충분한 공간이 필요합니다.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [8]:
# 크로마 패키지 설치
!pip install langchain-chroma

  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.


In [9]:
from langchain_chroma import Chroma

# 벡터 스토어 준비
vectorstore = Chroma.from_documents(
    documents,
    embedding=hf_embeddings,
)

In [10]:
# 문자열 쿼리와의 유사도 검색
vectorstore.similarity_search("고양이")

[Document(id='eca21a7b-acfb-41ff-9d04-3945707dfd2b', metadata={'source': 'mammal-pets-doc'}, page_content='고양이는 독립심이 높은 반려동물이며, 자기만의 공간을 만끽하는 경우가 많습니다.'),
 Document(id='d6c647e6-e114-4d37-8ec2-394b104aa40d', metadata={'source': 'mammal-pets-doc'}, page_content='토끼는 사회적인 동물이며, 자유롭게 뛰어다닐 충분한 공간이 필요합니다.'),
 Document(id='e8403570-14ca-4f9f-8ed0-593c1ea1b152', metadata={'source': 'mammal-pets-doc'}, page_content='개는 충성심이 높고 친근하기 때문에 아주 멋진 반려동물입니다.'),
 Document(id='770c28d4-5797-4558-aae9-bc987848b2ec', metadata={'source': 'bird-pets-doc'}, page_content='앵무새는 사람의 말을 따라할 수 있는 영리한 새입니다.')]

In [11]:
# 문자열 쿼리와의 유사도로 비동기 검색
await vectorstore.asimilarity_search("고양이")

[Document(id='eca21a7b-acfb-41ff-9d04-3945707dfd2b', metadata={'source': 'mammal-pets-doc'}, page_content='고양이는 독립심이 높은 반려동물이며, 자기만의 공간을 만끽하는 경우가 많습니다.'),
 Document(id='d6c647e6-e114-4d37-8ec2-394b104aa40d', metadata={'source': 'mammal-pets-doc'}, page_content='토끼는 사회적인 동물이며, 자유롭게 뛰어다닐 충분한 공간이 필요합니다.'),
 Document(id='e8403570-14ca-4f9f-8ed0-593c1ea1b152', metadata={'source': 'mammal-pets-doc'}, page_content='개는 충성심이 높고 친근하기 때문에 아주 멋진 반려동물입니다.'),
 Document(id='770c28d4-5797-4558-aae9-bc987848b2ec', metadata={'source': 'bird-pets-doc'}, page_content='앵무새는 사람의 말을 따라할 수 있는 영리한 새입니다.')]

In [12]:
# 문자열 쿼리와의 유사도로 검색 + 유사도 점수
vectorstore.similarity_search_with_score("고양이")

[(Document(id='eca21a7b-acfb-41ff-9d04-3945707dfd2b', metadata={'source': 'mammal-pets-doc'}, page_content='고양이는 독립심이 높은 반려동물이며, 자기만의 공간을 만끽하는 경우가 많습니다.'),
  0.7465862035751343),
 (Document(id='d6c647e6-e114-4d37-8ec2-394b104aa40d', metadata={'source': 'mammal-pets-doc'}, page_content='토끼는 사회적인 동물이며, 자유롭게 뛰어다닐 충분한 공간이 필요합니다.'),
  1.0053430795669556),
 (Document(id='e8403570-14ca-4f9f-8ed0-593c1ea1b152', metadata={'source': 'mammal-pets-doc'}, page_content='개는 충성심이 높고 친근하기 때문에 아주 멋진 반려동물입니다.'),
  1.026926875114441),
 (Document(id='770c28d4-5797-4558-aae9-bc987848b2ec', metadata={'source': 'bird-pets-doc'}, page_content='앵무새는 사람의 말을 따라할 수 있는 영리한 새입니다.'),
  1.1604771614074707)]

In [13]:
# 임베딩 쿼리와의 유사도 검색
embedding = hf_embeddings.embed_query("고양이")
vectorstore.similarity_search_by_vector(embedding)

[Document(id='eca21a7b-acfb-41ff-9d04-3945707dfd2b', metadata={'source': 'mammal-pets-doc'}, page_content='고양이는 독립심이 높은 반려동물이며, 자기만의 공간을 만끽하는 경우가 많습니다.'),
 Document(id='d6c647e6-e114-4d37-8ec2-394b104aa40d', metadata={'source': 'mammal-pets-doc'}, page_content='토끼는 사회적인 동물이며, 자유롭게 뛰어다닐 충분한 공간이 필요합니다.'),
 Document(id='e8403570-14ca-4f9f-8ed0-593c1ea1b152', metadata={'source': 'mammal-pets-doc'}, page_content='개는 충성심이 높고 친근하기 때문에 아주 멋진 반려동물입니다.'),
 Document(id='770c28d4-5797-4558-aae9-bc987848b2ec', metadata={'source': 'bird-pets-doc'}, page_content='앵무새는 사람의 말을 따라할 수 있는 영리한 새입니다.')]

### 검색기 준비

In [14]:
# 검색기 준비
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

In [15]:
# 검색기 실행
retriever.batch(["고양이", "물고기"])

[[Document(id='eca21a7b-acfb-41ff-9d04-3945707dfd2b', metadata={'source': 'mammal-pets-doc'}, page_content='고양이는 독립심이 높은 반려동물이며, 자기만의 공간을 만끽하는 경우가 많습니다.')],
 [Document(id='9d46d29d-4612-4164-9219-dbf622385f60', metadata={'source': 'fish-pets-doc'}, page_content='금붕어는 비교적 간단하게 키울 수 있기 때문에 초심자에게 인기가 많은 반려동물입니다.')]]

### 검색 증강 생성 구현

In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM 준비
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",
)

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 준비
message = """
제공된 컨텍스트만을 사용해서 이 질문에 답변해주세요.

{question}

Context:
{context}
"""

prompt_template = ChatPromptTemplate.from_messages([("human", message)])

In [18]:
# 검색 증강 생성 체인 준비
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
)

In [19]:
# 질의 응답
response = rag_chain.invoke("고양이에 관해 알려주십시오.")
print(response.content)

제공된 문서에 따르면, 고양이는 독립심이 높은 반려동물이며, 자기만의 공간을 좋아합니다.


### 문서 불러오기

In [20]:
# unstructured 패키지 설치
!pip install unstructured

In [23]:
# nltk 리소스 추가

import nltk

required_resources = [
            'punkt',
            'stopwords',
            'averaged_perceptron_tagger_eng',
            'punkt_tab'
        ]

for resource in required_resources:
    try:
        nltk.data.find(resource)
    except LookupError:
        nltk.download(resource)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [24]:
from langchain.document_loaders import DirectoryLoader

# 문서 불러오기
loader = DirectoryLoader("./data/")
documents = loader.load()

In [25]:
from langchain_chroma import Chroma

# 벡터 스토어 준비
vectorstore = Chroma.from_documents(
    documents,
    embedding=hf_embeddings,
)

In [26]:
# 검색기 준비
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2},
)

In [27]:
# 검색 증강 생성 준비
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
)

In [28]:
# 질의 응답
response = rag_chain.invoke("은비가 알게된 비밀은?")
print(response.content)

제공된 텍스트에 따르면, 은비가 알게 된 비밀은 **아크 코퍼레이션의 불법적인 실험**에 대한 증거입니다.
